In [69]:
# https://docs.getdbt.com/reference/configs-and-properties

In [122]:
import csv, yaml, sys, re, json, ruamel.yaml, os
from openpyxl import load_workbook
yaml = ruamel.yaml.YAML()
yaml.indent(mapping=2, sequence=4, offset=2)

In [99]:
# This function forces the YAML to output a python list as an array
# rather than an ordered list.
def seq(l):
    s = ruamel.yaml.comments.CommentedSeq(l)
    s.fa.set_flow_style()
    return s

In [164]:
## Schema File Generator
# Currently using an exported file, but could connect directly to database to pull infornmation_schema.columns.

def get_named_cell_location(cellname):
    return list(wb.defined_names[cellname].destinations)[0][1].replace('$','')

def write_to_cell(workbook, cell, input_data):
    workbook.worksheets[0][cell].value = input_data


all_tables_list = set([i.split(',')[1] for i in f.split('\n') if len(i) >= 2])
table_list = [j for j in all_tables_list if 'INT_' in j or 'RPT_' in j or 'FACT_' in j or 'DIM_' in j or 'STG_' in j] 
for table_name in table_list:
    wb = load_workbook(filename = 'documentation.adamr.template.xlsx')
    schema_name = [i.split(',') for i in f.split('\n')[1:] if len(i)>1 and i.split(',')[1] == table_name][0][0]
    table_meta = {}
    table_tags = []

    owner_tag = "@adamr"
    if owner_tag:
        table_meta["owner"] = owner_tag

    is_hourly = True
    if is_hourly:
        table_tags.append('hourly')
        
    f = open('schemas_2023_05_26.csv','r').read()
    table_name_lower = table_name.lower()
    l = [i.split(',') for i in f.split('\n') if table_name in i]
    write_to_cell(wb, get_named_cell_location('table_name'), table_name)
    write_to_cell(wb, get_named_cell_location('table_meta'), str(table_meta).replace("'",'"'))
    if is_hourly:
        write_to_cell(wb, get_named_cell_location('table_tags'), ','.join(table_tags))
        
    if 'INT_' in table_name or 'RPT_' in table_name:
        write_to_cell(wb,get_named_cell_location('table_full_refresh'), True)
        
    for i in range(0,len(l)):
        i_12 = i+12
        write_to_cell(wb,f'A{i_12}',l[i][2])
        write_to_cell(wb,f'B{i_12}',l[i][3])
    
    if schema_name not in os.listdir('docfiles'):
            os.mkdir(f'docfiles/{schema_name}')
    
    wb.save(f'docfiles/{schema_name}/documentation.adamr.{table_name_lower}.xlsx')

In [165]:
# load workbook
for schema_upper in [i for i in os.listdir('docfiles') if i != 'OUTPUT']: 
    output = {}
    models=[]
    schema = schema_upper.lower()
    for file in os.listdir(f'docfiles/{schema_upper}'):
        wb = load_workbook(filename = f'docfiles/{schema_upper}/{file}')

        ws = wb.worksheets[0]
        table_data={}
        table_inputs = [(j[0].value.lower(), j[1].value) for j in ws['A2:B8']]
        for i in table_inputs:
            k=i[0] # key
            v=i[1] # value
            if v is None:
                continue
            if k in ['name','description']:
                table_data[k] = v
            else:
                if 'config' not in table_data:
                    table_data['config']={}

                if k == 'meta':
                    table_data['config'][k]=json.loads(v)
                elif k == 'tags':
                    table_data['config'][k]=v.split(',')
                else:
                    table_data['config'][k]=v

        columns=[]
        for item in ws['A12:{:s}{:d}'.format([i.coordinate for i in ws[11]][-1][0],ws.max_row)]:
            column_cells = [i for i in ws[11]]
            col_data={}
            for ix, column in enumerate(column_cells):
                if item[ix].value is None:
                    continue
                column_name = column.value.lower()
                if column_name == 'tests_standard':
                    col_data['tests'] = item[ix].value.split(',')
                elif column_name == 'tests_accepted_values':
                    if 'tests' not in col_data:
                        col_data['tests'] = []

                    obj = {}
                    val = {}
                    val['values'] = seq(item[ix].value.split(','))
                    obj['accepted_values'] = val
                    col_data['tests'].append(obj)
                elif column_name == 'meta':        
                    col_data['meta']={}
                    for i in item[5].value.split('|'):
                        for k,v in json.loads(i).items():
                            col_data['meta'][k]=v
                else:                
                    col_data[column.value.lower()] = item[ix].value 

            columns.append(col_data)

        table_data['columns']=columns
        models.append(table_data)

    output['version'] = 2
    output['models'] = models
    with open(f'docfiles/OUTPUT/documentation.adamr.{schema}.yml','w') as o:
        yaml.dump(output,o)

In [ ]:
    table_data['name']=ws['B1'].value.lower()
    table_data['description']=ws['B2'].value
    table_data['config']={}
    table_data['config']['meta']={}
    for i in ws['B3'].value.split('|'):
        meta_items = i.split(':')
        table_data['config']['meta'][meta_items[0]]=meta_items[1].strip()

In [297]:
[(j[0].value.lower(), j[1].value) for ix, j in enumerate(wb.worksheets[0]['A2:B8'])]

[('name', 'rpt_lab_event_log'),
 ('description',
  'A Key-Value Datastore with metadata, with data for all events related to the CLIA laboratory process'),
 ('meta', '{owner: @adamr}'),
 ('tags', 'hourly,test'),
 ('enabled', 'true'),
 ('full refresh', 'true'),
 ('grants', None)]